### Parsing Market Data for the regression analysis


In [1]:
import yfinance as yf
import pandas as pd

# Function to define asset tickers
def get_asset_tickers():
    return {
        '10Y_Treasury_Yield': '^TNX',    # 10-year Treasury yield
        '2Y_Treasury_Yield': '^IRX',     # 2-year Treasury yield (using IRX as a proxy for short-term yields)
        '1Y_Treasury_Yield': '^FVX',     # 1-year Treasury yield
        'USD_Index': 'DX-Y.NYB',         # US Dollar Index
        'Russell1000_Growth': 'IWF',     # iShares Russell 1000 Growth ETF
        'Russell1000_Value': 'IWD'       # iShares Russell 1000 Value ETF
    }

# Function to download data from Yahoo Finance
def download_historical_data(tickers, start_date, end_date):
    return yf.download(list(tickers.values()), start=start_date, end=end_date)

# Function to clean and process the downloaded data
def process_close_prices(data, tickers):
    close_prices = data['Close'].copy()
    # Rename columns to match the descriptive tickers
    close_prices.columns = [name for col in close_prices.columns for name, ticker in tickers.items() if ticker == col]
    # Reorder the columns based on the order of the tickers dictionary
    close_prices = close_prices[list(tickers.keys())]
    return close_prices

# Function to calculate the yield and ETF spreads
def calculate_spreads(df):
    df['10Y-2Y_Yield_Spread'] = df['10Y_Treasury_Yield'] - df['2Y_Treasury_Yield']
    df['Growth-Value_Spread'] = df['Russell1000_Growth'] - df['Russell1000_Value']
    return df

# Function to calculate percentage changes
def calculate_percentage_changes(df):
    pct_change_df = df.pct_change() * 100  # Calculate day-over-day percentage change
    pct_change_df.columns = [f'{col}_Pct_Change' for col in df.columns]  # Rename columns with '_Pct_Change' suffix
    return pct_change_df

# Function to combine original data and percentage changes
def combine_data_with_pct_change(df, pct_change_df):
    combined_df = pd.concat([df, pct_change_df], axis=1)  # Concatenate the original and percentage change data
    return combined_df

# Main function to run the process
def main():
    # Define tickers and dates
    tickers = get_asset_tickers()
    start_date = "2012-01-01"
    end_date = "2024-10-10"
    
    # Download and process historical data
    historical_data = download_historical_data(tickers, start_date, end_date)
    close_prices_df = process_close_prices(historical_data, tickers)
    
    # Calculate spreads and percentage changes
    close_prices_df = calculate_spreads(close_prices_df)
    pct_change_df = calculate_percentage_changes(close_prices_df)
    
    # Combine the close prices and percentage changes into one DataFrame
    combined_df = combine_data_with_pct_change(close_prices_df, pct_change_df)
    
    # Reset the index for readability
    combined_df.reset_index(inplace=True)
    return combined_df

combined_df = main()


[**********************50%%                      ]  3 of 6 completed

/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex cons

[*********************100%%**********************]  6 of 6 completed


/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/aidosutebayev/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_36416/1700205495.py:36: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  pct_change_df = df.pct_change() * 100  # Calculate day-over-day percentage change


In [2]:
combined_df.head()

,Date,10Y_Treasury_Yield,2Y_Treasury_Yield,1Y_Treasury_Yield,USD_Index,Russell1000_Growth,Russell1000_Value,10Y-2Y_Yield_Spread,Growth-Value_Spread,10Y_Treasury_Yield_Pct_Change,2Y_Treasury_Yield_Pct_Change,1Y_Treasury_Yield_Pct_Change,USD_Index_Pct_Change,Russell1000_Growth_Pct_Change,Russell1000_Value_Pct_Change,10Y-2Y_Yield_Spread_Pct_Change,Growth-Value_Spread_Pct_Change
0,2012-01-03,1.960,0.005,0.886,79.610001,58.630001,64.529999,1.955,-5.899998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-04,1.995,0.010,0.889,80.089996,58.720001,64.510002,1.985,-5.790001,1.785713,100.000000,0.338603,0.602934,0.153505,-0.030988,1.534525,-1.864353
2,2012-01-05,1.993,0.010,0.878,80.940002,58.880001,64.790001,1.983,-5.910000,-0.100249,0.000000,-1.237343,1.061314,0.272479,0.434039,-0.100754,2.072520
3,2012-01-06,1.961,0.015,0.856,81.239998,58.810001,64.610001,1.946,-5.799999,-1.605623,50.000000,-2.505696,0.370639,-0.118885,-0.277821,-1.865863,-1.861262
4,2012-01-09,1.960,0.005,0.841,81.050003,58.840000,64.820000,1.955,-5.980000,-0.050991,-66.666667,-1.752335,-0.233869,0.051010,0.325026,0.462491,3.103454


Remove unnecessary columns

In [3]:
columns = ['Date', '1Y_Treasury_Yield_Pct_Change', 'USD_Index_Pct_Change', '10Y-2Y_Yield_Spread_Pct_Change', 'Growth-Value_Spread_Pct_Change']
combined_df = combined_df[columns].dropna()
combined_df.head()

,Date,1Y_Treasury_Yield_Pct_Change,USD_Index_Pct_Change,10Y-2Y_Yield_Spread_Pct_Change,Growth-Value_Spread_Pct_Change
1,2012-01-04,0.338603,0.602934,1.534525,-1.864353
2,2012-01-05,-1.237343,1.061314,-0.100754,2.072520
3,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
4,2012-01-09,-1.752335,-0.233869,0.462491,3.103454
5,2012-01-10,1.545773,-0.308452,0.358054,1.839539


In [17]:


# Get the list of trading dates from df_combined
trading_dates = pd.to_datetime(combined_df['Date']).to_list()

# Create a list to store average market data
average_market_data = []

def get_next_trading_days(filing_date, trading_dates, num_days=3):
    filing_date = pd.to_datetime(filing_date)

    # Find the index of the filing date
    if filing_date in trading_dates:
        filing_idx = trading_dates.index(filing_date)
    else:
        # Find the index of the next trading date
        next_dates = [date for date in trading_dates if date > filing_date]
        if not next_dates:
            return []  # Return empty list if there are no dates after filing_date
        filing_idx = trading_dates.index(min(next_dates))

    # Return the next `num_days` trading days
    return trading_dates[filing_idx : filing_idx  + num_days]

# Iterate through fed_df rows
for _, row in combined_df.iterrows():
    filing_date = row['Date']

    # Get the next 3 trading days after the filing date
    next_trading_days = get_next_trading_days(filing_date, trading_dates, num_days=2)

    if not next_trading_days:
        print(f"No trading days found after filing date {filing_date}.")
        average_market_data.append(pd.Series())  # Append an empty series if no trading days found
        continue  # Skip to the next iteration

    # Calculate the average market data for the next trading days
    filtered_df = combined_df[combined_df['Date'].isin(next_trading_days)]

    if not filtered_df.empty:
        average_data = filtered_df.mean()
        average_market_data.append(pd.Series())  # Append empty series if no data

# Create a DataFrame for the average market data
average_market_df = pd.DataFrame(average_market_data)

# Combine fed_df with average market data
combined_df = pd.concat([combined_df.reset_index(drop=True), average_market_df.reset_index(drop=True)], axis=1)

# Display the combined DataFrame
print(combined_df)


           Date  1Y_Treasury_Yield_Pct_Change  USD_Index_Pct_Change  \
0    2012-01-04                      0.338603              0.602934   
1    2012-01-05                     -1.237343              1.061314   
2    2012-01-06                     -2.505696              0.370639   
3    2012-01-09                     -1.752335             -0.233869   
4    2012-01-10                      1.545773             -0.308452   
...         ...                           ...                   ...   
3208 2024-10-03                      2.307909              0.304876   
3209 2024-10-04                      4.896836              0.519658   
3210 2024-10-07                      1.468660              0.019513   
3211 2024-10-08                     -0.129229              0.009754   
3212 2024-10-09                      1.138714              0.370548   

      10Y-2Y_Yield_Spread_Pct_Change  Growth-Value_Spread_Pct_Change  
0                           1.534525                       -1.864353  
1    

In [22]:
combined_df.dropna(inplace=True)
combined_df=combined_df.rename(columns={"1Y_Treasury_Yield_Pct_Change": "1Y_Yield_Pct_Change_3day_avg",	"USD_Index_Pct_Change":	"DXY_Pct_Change_3day_avg", "10Y-2Y_Yield_Spread_Pct_Change": "10Y-2Y_Spread_Pct_Change_3day_avg",	"Growth-Value_Spread_Pct_Change": "Growth-Value_Spread_Pct_Change_3day_avg"})
combined_df.head()

,Date,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
0,2012-01-04,0.338603,0.602934,1.534525,-1.864353
1,2012-01-05,-1.237343,1.061314,-0.100754,2.072520
2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
3,2012-01-09,-1.752335,-0.233869,0.462491,3.103454
4,2012-01-10,1.545773,-0.308452,0.358054,1.839539


Save the dataframe to csv file

In [21]:
combined_df.to_csv('market_data.csv')